In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-flood/data_source/new_data/valid/flooding/11891573354.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/860024680.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/8334177117.png
/kaggle/input/data-flood/data_source/new_data/valid/flooding/7520315140.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/1827534805.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/6106111822.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/3855038967.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/3685111553.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/2269424912.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/407960551.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/4436863796.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/147160837.jpg
/kaggle/input/data-flood/data_source/new_data/valid/flooding/5355684661.jpg
/kaggle/input/

In [3]:
!pip install scikeras

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing import image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 57.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


2024-06-08 11:03:03.582650: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 11:03:03.582785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 11:03:03.673974: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Define paths
train_path = '/kaggle/input/data-flood/data_source/new_data/train'
valid_path = '/kaggle/input/data-flood/data_source/new_data/valid'
test_path = '/kaggle/input/data-flood/data_source/new_data/test'

# Create image data generators
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
    rotation_range=4,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
).flow_from_directory(
    directory=train_path, target_size=(224, 224), batch_size=10, shuffle=True
)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224, 224), batch_size=10, shuffle=True
)

# Convert data generators to numpy arrays
def generator_to_numpy(generator):
    x, y = [], []
    for i in range(len(generator)):
        xi, yi = generator[i]
        x.extend(xi)
        y.extend(yi)
    return np.array(x), np.array(y)

X_train, y_train = generator_to_numpy(train_batches)
X_valid, y_valid = generator_to_numpy(valid_batches)

def create_model(dropout_rate=0.2, learning_rate=1e-4, dense_units=128, activation='relu'):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(units=dense_units, activation=activation)(x)
    output = Dense(units=2, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    for layer in base_model.layers[-12:]:
        layer.trainable = True

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Wrap the model using KerasClassifier from scikeras
model = KerasClassifier(model=create_model, verbose=2)

# Define the parameter grid with additional hyperparameters
param_dist = {
    'model__dropout_rate': [0.2, 0.3, 0.4, 0.5],
    'model__learning_rate': [1e-4, 1e-3, 1e-2],
    'model__dense_units': [64, 128, 256, 512],
    'model__activation': ['relu', 'tanh', 'sigmoid'],
    'batch_size': [10, 20, 30],
    'epochs': [5, 10, 15, 20]
}

# Perform Randomized Search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=20, cv=3, verbose=2, n_jobs=1)
random_search_result = random_search.fit(X_train, y_train, validation_data=(X_valid, y_valid))

# Extract best hyperparameters
best_params = random_search_result.best_params_
best_dropout_rate = best_params['model__dropout_rate']
best_learning_rate = best_params['model__learning_rate']
best_dense_units = best_params['model__dense_units']
best_activation = best_params['model__activation']
best_batch_size = best_params['batch_size']
best_epochs = best_params['epochs']

# Train the final model with the best hyperparameters
best_model = create_model(
    dropout_rate=best_dropout_rate,
    learning_rate=best_learning_rate,
    dense_units=best_dense_units,
    activation=best_activation
)
best_model.fit(X_train, y_train, epochs=best_epochs, validation_data=(X_valid, y_valid), batch_size=best_batch_size, verbose=2)

Found 3693 images belonging to 2 classes.
Found 1054 images belonging to 2 classes.
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Epoch 1/10


W0000 00:00:1717847452.932170     140 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1717847460.543743     139 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1717847464.015724     138 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


83/83 - 23s - 279ms/step - accuracy: 0.5601 - loss: 1.6086 - val_accuracy: 0.6309 - val_loss: 0.6663
Epoch 2/10


W0000 00:00:1717847467.366568     137 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


83/83 - 3s - 40ms/step - accuracy: 0.5829 - loss: 0.7218 - val_accuracy: 0.3691 - val_loss: 1.6968
Epoch 3/10
83/83 - 3s - 40ms/step - accuracy: 0.5471 - loss: 1.0102 - val_accuracy: 0.6309 - val_loss: 0.7436
Epoch 4/10
83/83 - 3s - 40ms/step - accuracy: 0.5902 - loss: 0.7371 - val_accuracy: 0.6309 - val_loss: 0.6612
Epoch 5/10
83/83 - 3s - 40ms/step - accuracy: 0.5601 - loss: 0.8166 - val_accuracy: 0.6309 - val_loss: 0.8371
Epoch 6/10
83/83 - 3s - 41ms/step - accuracy: 0.5414 - loss: 0.8218 - val_accuracy: 0.6309 - val_loss: 0.6832
Epoch 7/10
83/83 - 3s - 40ms/step - accuracy: 0.5715 - loss: 0.7536 - val_accuracy: 0.3691 - val_loss: 0.7976
Epoch 8/10
83/83 - 3s - 40ms/step - accuracy: 0.5707 - loss: 0.7440 - val_accuracy: 0.3691 - val_loss: 0.7385
Epoch 9/10
83/83 - 3s - 40ms/step - accuracy: 0.5723 - loss: 0.7568 - val_accuracy: 0.3691 - val_loss: 0.8792
Epoch 10/10
83/83 - 3s - 40ms/step - accuracy: 0.5674 - loss: 0.8625 - val_accuracy: 0.3691 - val_loss: 1.4941


...

83/83 - 20s - 240ms/step - accuracy: 0.8245 - loss: 0.5417 - val_accuracy: 0.8292 - val_loss: 0.4015
Epoch 2/5


W0000 00:00:1717850090.905648     138 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


83/83 - 3s - 40ms/step - accuracy: 0.8899 - loss: 0.2724 - val_accuracy: 0.8824 - val_loss: 0.3254
Epoch 3/5
83/83 - 3s - 40ms/step - accuracy: 0.9127 - loss: 0.2242 - val_accuracy: 0.8292 - val_loss: 0.8179
Epoch 4/5
83/83 - 3s - 40ms/step - accuracy: 0.9216 - loss: 0.1780 - val_accuracy: 0.6414 - val_loss: 1.8603
Epoch 5/5
83/83 - 3s - 41ms/step - accuracy: 0.9318 - loss: 0.1611 - val_accuracy: 0.6746 - val_loss: 1.5049


W0000 00:00:1717850107.997011     137 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


42/42 - 4s - 96ms/step
[CV] END batch_size=30, epochs=5, model__activation=relu, model__dense_units=64, model__dropout_rate=0.5, model__learning_rate=0.01; total time=  44.7s


In [ ]:
def preprocess_image(file_path):
    img = image.load_img(file_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

def predict_on_folder(folder_path, model, labels, output_csv_path):
    results = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path) and file_name.endswith(('.jpg', '.png', '.jpeg', '.gif')):
            preprocessed_image = preprocess_image(file_path)
            predictions = model.predict(preprocessed_image)
            result = np.argmax(predictions)
            file_id = os.path.splitext(file_name)[0]
            results.append({'id': file_id, 'label': labels[result]})
    
    df_results = pd.DataFrame(results)
    df_results['label'] = df_results['label'].apply(lambda x: 1 if x == 'Flooding' else 0)
    df_results.to_csv(output_csv_path, index=False)

# Define labels
labels = ['Flooding','No_Flooding']

# Predict on test set and save results
img_folder_path = '/kaggle/input/testset-source/testset_source/new_testset'
output_csv_path = '/kaggle/working/predictions_16.csv'
predict_on_folder(img_folder_path, best_model, labels, output_csv_path)

print(f"Kết quả dự đoán đã được lưu vào {output_csv_path}")
